In [1]:
from sfm.data.sci_data.SFMDecTokenizer import SFMDecTokenizer
import torch
import os

/anaconda/envs/sfm2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-02-06 09:59:10,383] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
tokenizer_home = '/hai1/ds_dataset/llama2/llama-2-7b'
tokenizer = SFMDecTokenizer.from_pretrained(
    tokenizer_home,
    prot_spm_path='/blob/shufxi/data/scigpt/ur50bpe/bpe',
    dna_spm_path='/blob/shufxi/data/scigpt/dnabpe/bpe',
    rna_spm_path='/blob/shufxi/data/scigpt/rnabpe/bpe',
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LlamaTokenizer'. 
The class this function is called from is 'SFMDecTokenizer'.


You are using the default legacy behaviour of the <class 'sfm.data.sci_data.SFMDecTokenizer.SFMDecTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


[2024-02-06 09:59:12.457][INFO]: Loading protein sentencepiece model from /blob/shufxi/data/scigpt/ur50bpe/bpe.model and /blob/shufxi/data/scigpt/ur50bpe/bpe.vocab
[2024-02-06 09:59:12.515][INFO]: Loading DNA sentencepiece model from /blob/shufxi/data/scigpt/dnabpe/bpe.model and /blob/shufxi/data/scigpt/dnabpe/bpe.vocab
[2024-02-06 09:59:12.571][INFO]: Loading RNA sentencepiece model from /blob/shufxi/data/scigpt/rnabpe/bpe.model and /blob/shufxi/data/scigpt/rnabpe/bpe.vocab
[2024-02-06 09:59:12.894][INFO]: Tokenizer has 40014 tokens


In [10]:
tokenizer.tokenize('<rna>AUCGAAA</rna>')

['<rna>', '<r>AUC', '<r>GAAA', '</rna>']

Bad pipe message: %s [b'\xaf\xd3\x85\xfdC\xa1\xf4\xc5\x1d\xed\xb3\xce\x0fA\xdc\x86\x99\x88 \xf5\x11U\x91A\xd9\xf0)\x91\x1e\xd8g\r;_\x01P\xf8\x19t\x08\xaeS\x8e\xbem\xbdKPc\xff\xc9\x00\x08\x13\x02\x13\x03']
Bad pipe message: %s [b'\x06\xd0\x0c\xcc\x11\xf4\xd6\xb0z\x95\x9b\xc95\xe8\x1f\xc80\x00 \xe0>=\xbd!G\x96\xc7\xa0eYRyF\xb5\x1e\xb4\xf1\x1c\x1f\xf5==\xe4\xfd\x16\xef\xe1\x88\x06\x17\xe4\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17']
Bad pipe message: %s [b'\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127', b'.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00']
Bad pipe message: %s [b'\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01']
Bad pipe message: %s [b'si\xd0\x8d4H:\x12Q\x01\xf7#\x1d\xfb

In [25]:
s = tokenizer.tokenize('The <protein>EMMFEQTFKNID</protein> and the compound <mol>CCC</mol> can interact with each other.')
print(' '.join(s))

▁The <protein> <a>E <a>M <a>M <a>FEQ <a>TFK <a>NID </protein> ▁and ▁the ▁comp ound <mol> <m>C <m>C <m>C </mol> ▁can ▁interact ▁with ▁each ▁other .


In [26]:
from transformers import AutoTokenizer, AutoModelForCausalLM
# ckpt_home = '/hai1/shufxi/scigpt/7bv2/stageB/global_step26999/'

ckpt_home = '/blob/shufxi/scigpt/7bv2/stageB/finetune/pistachio/global_step1999'

def show_ckpt(name, ckpt):
    for k, v in ckpt.items():
        if 'dummy' not in k:
            print(name, k, v.shape)

model = AutoModelForCausalLM.from_pretrained(tokenizer_home)

model_dict = model.state_dict()
ckpt_dict = {}
layer0 = torch.load(os.path.join(ckpt_home, "layer_00-model_states.pt"), map_location=torch.device("cpu"))
ckpt_dict['model.embed_tokens.weight'] = layer0['embed_tokens.weight']
show_ckpt('layer0', layer0)

for l in range(0, 32):
    l_index = str(l + 1).zfill(2)
    layer = torch.load(os.path.join(ckpt_home, f"layer_{l_index}-model_states.pt"), map_location=torch.device("cpu"))
    show_ckpt(l_index, layer)
    for k in layer:
        if "dummy" in k or 'rotary_emb' in k:
            continue
        ckpt_dict[f"model.layers.{l}.{k}"] = layer[k]
layer = torch.load(os.path.join(ckpt_home, "layer_33-model_states.pt"), map_location=torch.device("cpu"))
show_ckpt(33, layer)
ckpt_dict["model.norm.weight"] = layer["norm.weight"]

layer = torch.load(os.path.join(ckpt_home, "layer_34-model_states.pt"), map_location=torch.device("cpu"))
show_ckpt(33, layer)
ckpt_dict["lm_head.weight"] = layer["lm_head.weight"]
model_dict.update(ckpt_dict)

model.resize_token_embeddings(len(tokenizer))
model.load_state_dict(model_dict)

Loading checkpoint shards: 100%|██████████| 2/2 [00:21<00:00, 10.88s/it]


layer0 embed_tokens.weight torch.Size([38991, 4096])
01 self_attn.q_proj.weight torch.Size([4096, 4096])
01 self_attn.k_proj.weight torch.Size([4096, 4096])
01 self_attn.v_proj.weight torch.Size([4096, 4096])
01 self_attn.o_proj.weight torch.Size([4096, 4096])
01 mlp.gate_proj.weight torch.Size([11008, 4096])
01 mlp.up_proj.weight torch.Size([11008, 4096])
01 mlp.down_proj.weight torch.Size([4096, 11008])
01 input_layernorm.weight torch.Size([4096])
01 post_attention_layernorm.weight torch.Size([4096])
02 self_attn.q_proj.weight torch.Size([4096, 4096])
02 self_attn.k_proj.weight torch.Size([4096, 4096])
02 self_attn.v_proj.weight torch.Size([4096, 4096])
02 self_attn.o_proj.weight torch.Size([4096, 4096])
02 mlp.gate_proj.weight torch.Size([11008, 4096])
02 mlp.up_proj.weight torch.Size([11008, 4096])
02 mlp.down_proj.weight torch.Size([4096, 11008])
02 input_layernorm.weight torch.Size([4096])
02 post_attention_layernorm.weight torch.Size([4096])
03 self_attn.q_proj.weight torch.Size

<All keys matched successfully>

In [27]:
model = model.cuda()

In [29]:
input_ids = tokenizer('<product>C(\C(N[C@H]1CN2CCC1CC2)=O)=C/c1cccc(Br)c1</product><reactants>', return_tensors="pt").input_ids.cuda()
output = model.generate(
    input_ids,
    num_beams=4,
    max_new_tokens=300,
    num_return_sequences=4,
    return_dict_in_generate=True,
    output_scores=True
)

for i in range(4):
    mol = tokenizer.decode(output.sequences[i])
    mol = mol.replace(' <m>', '')
    print(mol)
    print(output.sequences[i])
    print(output.sequences_scores[i].item())
    print("")

<s> <product>C(\C(N[C@H]1CN2CCC1CC2)=O)=C/c1cccc(Br)c1 </product> <reactants>C(\)cccc(Br)c1ccccc1)ccc1ccccc1)ccc1ccccc1)ccc1ccccc1)ccc1ccccc1)ccc1ccccc1)ccc1ccccc1)ccc1ccccc1)ccc1ccccc1)ccc1ccccc1)ccc1ccccc1)ccc1ccccc1)ccc1ccccc1)ccc1ccccc1)ccc1ccccc1)ccc1ccccc1)ccc1ccccc1)ccc1ccccc1)ccc1ccccc1)ccc1ccccc1)ccc1ccccc1)ccc1ccccc1)ccc1ccccc1)ccc1ccccc1)ccc1ccccc1)ccc1ccccc1)ccc1c
tensor([    1, 32017, 32128, 32129, 32155, 32128, 32129, 32135, 32143, 32131,
        32128, 32135, 32134, 32128, 32128, 32128, 32131, 32128, 32128, 32134,
        32130, 32133, 32132, 32130, 32133, 32128, 32145, 32127, 32131, 32127,
        32127, 32127, 32127, 32129, 32151, 32130, 32127, 32131, 32018, 32019,
        32128, 32129, 32155, 32130, 32127, 32127, 32127, 32127, 32129, 32151,
        32130, 32127, 32131, 32127, 32127, 32127, 32127, 32127, 32131, 32130,
        32127, 32127, 32127, 32131, 32127, 32127, 32127, 32127, 32127, 32131,
        32130, 32127, 32127, 32127, 32131, 32127, 32127, 32127, 32127, 3212

Bad pipe message: %s [b"\xfbR\xcaiy\x078<\x92f\x8b%\xfc\xe7/\xb1r\x08\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0\x11\x00\x96\x00\x05\x00\xff\x01\x00\x00j\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x000\x00.\x04\x03\x05\x03", b'\x08\x07\x08\x08\x08', b'\n\x08\x0b\x08\x04\x08\x05\x08']
Bad pipe message: %s [b'\x01\x05\x01\x06\x01']
Bad pipe message: %s [b'\x02\x03', b'\x02\x01', b'\x02\x02']
Bad pipe message: %s [b'\xcd\xda\xb43\x9d\x91\xbaU\nh\x07\x971^Y{\x9eV\x00\x00\xa6\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa

In [22]:
tokenizer.encode("<product>C</product><reactants>C.C</reactants>", add_special_tokens=False)

[32017, 32128, 32018, 32019, 32128, 32148, 32128, 32020]

Bad pipe message: %s [b'\xc5\xbb\xa1y}b\xce\xbes"\xdal\xd4w\x8e,]\x0b\x00\x00\xa6\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0', b"`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0s\xc0w\x00\xc4\x00\xc3\xc0#\xc0'\x00g\x00@\xc0r\xc0v\x00\xbe\x00\xbd\xc0\n\xc0\x14\x009\x008\x00\x88\x00\x87\xc0\t\xc0\x13\x003\x002\x00\x9a\x00\x99\x00E\x00D\xc0\x07\xc0\x11\xc0\x08\xc0\x12\x00\x16\x00\x13\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c"]
Bad pipe message: %s [b'\xd0G\xa1\xa9\x94\xf9H69[\xc9G*\r-@8\x92\x00\x00>\xc0\x14\xc0\n\x009\x008\x007\x006\xc0\x0f\xc0\x05\x005\xc0\x13\xc0\t\x003\x002\x001\x000\xc0\x0e\xc0\x04\x00/\x00\x9a\x00\x99\x00\x98\x00\x97\x00\x96\x00\x07\xc0\x11\xc0\x07\xc0\x0c\xc0\x02\x00\x05\x00\x04\x00\xff\x02\x01\x00\x00C\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01']
Bad pipe message: %s [b'\n']
Bad pipe message: %s [b'\xfe\x88\x972r\xea\xed

In [5]:
input_ids = tokenizer('A drug for SARS-CoV-2 is <mol>', return_tensors="pt").input_ids.cuda()
output = model.generate(
    input_ids,
    num_beams=4,
    max_new_tokens=300,
    num_return_sequences=4,
    return_dict_in_generate=True,
    output_scores=True
)

for i in range(4):
    s = tokenizer.decode(output.sequences[i])
    mol = s[s.find('<mol>') + len('<mol>'):s.find('</mol>')]
    mol = mol.replace('<m>', '').replace(' ', '')
    print(mol, output.sequences_scores[i].item())

Cc1cccc(C)c1OCC(=O)N[C@@H](Cc1ccccc1)[C@@H](O)C[C@H](Cc1ccccc1)NC(=O)[C@H](C(C)C)N1CCCNC1=O -0.037261441349983215
COc1ccc2c(O[C@@H]3C[C@H]4C(=O)N[C@]5(C(=O)NS(=O)(=O)C6CC6)C[C@@H]5/C=C\CCCCN(C)C(=O)[C@@H]4C3)cc(-c3nc(C(C)C)cs3)nc2c1C -0.04674167558550835
COc1ccc2c(O[C@@H]3C[C@H]4C(=O)N[C@]5(C(=O)NS(=O)(=O)C6CC6)C[C@H]5/C=C\CCCCN(C)C(=O)[C@@H]4C3)cc(-c3nc(C(C)C)cs3)nc2c1C -0.054373256862163544
CC[NH+](CCO)CCCC(C)Nc1cc[nH+]c2cc(Cl)ccc12 -0.057832781225442886


In [1]:
data = []
with open('/home/shufxi/pubchem_bp.csv', 'r') as f:
    for line in f:
        items = line.strip().split(',')
        if '.' in items[0]:
            continue
        if items[1] == 'bp':
            continue
        data.append((items[0], float(items[1])))
data[:3]

[('c1ccoc1', 41.50445005391703),
 ('ClCCl', 40.16701092533697),
 ('CCC=O', 47.65522297265193)]

In [2]:
'COCOC' in data

False

In [4]:
data

[('c1ccoc1', 41.50445005391703),
 ('ClCCl', 40.16701092533697),
 ('CCC=O', 47.65522297265193),
 ('FC(F)OC(Cl)C(F)(F)F', 49.067838503088296),
 ('FC(F)(F)C(Cl)Br', 49.660424521001175),
 ('CCCN', 47.28948020617565),
 ('CI', 42.46094566320693),
 ('C1=CCC=C1', 40.121684454299725),
 ('COCCl', 45.3855737491495),
 ('CC(C)(C)N', 43.060588594112346),
 ('COCOC', 41.738317772769165),
 ('C=CCCl', 44.7251440297772),
 ('CCC(C)(C)C', 49.58675683847429),
 ('CON', 45.11570766264106),
 ('FC(F)(Cl)C(F)(Cl)Cl', 47.070521229944575),
 ('C[Pb](C)(C)C', 42.88601516718609),
 ('CC[Hg]Cl', 48.341138692739946),
 ('CCCCl', 46.25771085829854),
 ('FC(F)(Br)C(F)(F)Br', 47.549482837282994),
 ('C=CC=CC', 42.873653402357775),
 ('FC(F)(F)C(Cl)(Cl)Cl', 44.15316095324488),
 ('C1=CCCC1', 44.24730179484448),
 ('FC(F)(Cl)CCl', 45.875647966602685),
 ('CC(=O)Cl', 49.21110194823143),
 ('C=COCC(F)(F)F', 43.89943654128263),
 ('O=S(=O)=O', 43.600872341937816),
 ('C1COC1', 46.834998786666574),
 ('ClSCl', 47.29371435397121)]

In [12]:
import random
random.seed(42)
random.shuffle(data)

In [33]:
def make_prompt(k=10, new_bp=40):
    ret = ''
    for item in random.choices(data, k=k):
        ret += f'A moleclue with boiling point {item[1]:.4f} is <mol>{item[0]}</mol>. '
    
    ret += f'A moleclue with boiling point {new_bp:.4f} is <mol>'

    return ret

prompt = make_prompt(k=20, new_bp=40 + 10*random.random())
print(prompt)
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
output = model.generate(input_ids, do_sample=True, top_p=0.9, max_new_tokens=100, num_return_sequences=1)
# output = model.generate(input_ids, num_beams=4, max_new_tokens=100, num_return_sequences=4)
for s in output:
    s = tokenizer.decode(s)
    print(s)
    mol = s[s.rfind('<mol>') + len('<mol>'):s.rfind('</mol>')]
    mol = mol.replace('<m>', '').replace(' ', '')
    print(mol)

A moleclue with boiling point 48.3411 is <mol>CC[Hg]Cl</mol>. A moleclue with boiling point 47.0705 is <mol>FC(F)(Cl)C(F)(Cl)Cl</mol>. A moleclue with boiling point 42.8860 is <mol>C[Pb](C)(C)C</mol>. A moleclue with boiling point 48.3411 is <mol>CC[Hg]Cl</mol>. A moleclue with boiling point 43.0606 is <mol>CC(C)(C)N</mol>. A moleclue with boiling point 42.8737 is <mol>C=CC=CC</mol>. A moleclue with boiling point 43.6009 is <mol>O=S(=O)=O</mol>. A moleclue with boiling point 45.3856 is <mol>COCCl</mol>. A moleclue with boiling point 42.8860 is <mol>C[Pb](C)(C)C</mol>. A moleclue with boiling point 48.3411 is <mol>CC[Hg]Cl</mol>. A moleclue with boiling point 42.8860 is <mol>C[Pb](C)(C)C</mol>. A moleclue with boiling point 41.7383 is <mol>COCOC</mol>. A moleclue with boiling point 42.8860 is <mol>C[Pb](C)(C)C</mol>. A moleclue with boiling point 45.1157 is <mol>CON</mol>. A moleclue with boiling point 48.3411 is <mol>CC[Hg]Cl</mol>. A moleclue with boiling point 48.3411 is <mol>CC[Hg]C

<s>A moleclue with boiling point 48.3411 is <mol> <m>C <m>C <m>[Hg] <m>Cl </mol> . A moleclue with boiling point 47.0705 is <mol> <m>F <m>C <m>( <m>F <m>) <m>( <m>Cl <m>) <m>C <m>( <m>F <m>) <m>( <m>Cl <m>) <m>Cl </mol> . A moleclue with boiling point 42.8860 is <mol> <m>C <m>[Pb] <m>( <m>C <m>) <m>( <m>C <m>) <m>C </mol> . A moleclue with boiling point 48.3411 is <mol> <m>C <m>C <m>[Hg] <m>Cl </mol> . A moleclue with boiling point 43.0606 is <mol> <m>C <m>C <m>( <m>C <m>) <m>( <m>C <m>) <m>N </mol> . A moleclue with boiling point 42.8737 is <mol> <m>C <m>= <m>C <m>C <m>= <m>C <m>C </mol> . A moleclue with boiling point 43.6009 is <mol> <m>O <m>= <m>S <m>( <m>= <m>O <m>) <m>= <m>O </mol> . A moleclue with boiling point 45.3856 is <mol> <m>C <m>O <m>C <m>Cl </mol> . A moleclue with boiling point 42.8860 is <mol> <m>C <m>[Pb] <m>( <m>C <m>) <m>( <m>C <m>) <m>C </mol> . A moleclue with boiling point 48.3411 is <mol> <m>C <m>C <m>[Hg] <m>Cl </mol> . A moleclue with boiling point 42.8860 is

: 

In [6]:
input_ids = tokenizer('A protein that fold like a circle is <protein>', return_tensors="pt").input_ids.cuda()
n = 10
output = model.generate(
    input_ids,
    num_beams=n,
    # top_p=0.85,
    # do_sample=True,
    max_new_tokens=100,
    num_return_sequences=n,
    return_dict_in_generate=True,
    output_scores=True
)

for i in range(n):
    s = tokenizer.decode(output.sequences[i])
    mol = s[s.find('<protein>') + len('<protein>'):s.find('</protein>')]
    mol = mol.replace('<a>', '').replace(' ', '')
    print(mol)

MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGGMQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGGMQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGGMQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLE
MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGGMQIFVKTLTGKTITLEVEPSETIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGGMQIFVKTLTGKTITLEVEPSETIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGGMQIFVKTLTGKTITLEVEPSETIENVKAKIQDKEGIPPDQQRLIFAGKQLE
MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGGMQIFVKTLTGKTITLEVEPIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGGMQIFVKTLTGKTITLEVEPIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGGMQIFVKTLTGKTITLEVEPIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYN
MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGGMQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGGMQIFVKTL

In [11]:
input_ids = tokenizer("Design a protein with a structure resembling a common ordinal numeral 2: start with a horizontal segment, curve downwards and left in a semi-circle, then extend diagonally down right, ending with a horizontal base. Ensure structural stability and functionality while subtly mimicking this numeral's shape without direct replication. The result is <protein>", return_tensors="pt").input_ids.cuda()
n = 10
output = model.generate(
    input_ids,
    num_beams=n,
    # top_p=0.85,
    # do_sample=True,
    max_new_tokens=100,
    num_return_sequences=n,
    return_dict_in_generate=True,
    output_scores=True
)

for i in range(n):
    s = tokenizer.decode(output.sequences[i])
    mol = s[s.find('<protein>') + len('<protein>'):s.find('</protein>')]
    mol = mol.replace('<a>', '').replace(' ', '')
    print(mol)

MSSSPLSKKRRVSGPDPKPGSNCSPAQSVLSEVPSVPTNGMAKNGSEADIDEGLYSRQLYVLGHEAMKRLQTSSVLVSGLRGLGVEIAKNIILGGVKAVTLHDQGTAQWADLSSQFYLREEDIGKNRAEVSQPRLAELNSYVPVTAYTGPLVEDFLSGFQVVVLTNTPLEDQLRVGEFCHNRGIKLVVADTRGLFGQLFCDFGEEMILTDSNGEQPLSAMVSMVTKDNPGVVTCLDEARHGFESGDFVSFSEVQGMVELNGNQP
MTERPPSEAARSDPQLEGRDAAEASMAPPHLVLLNGVAKETSRAAAAEPPVIELGARGGPGGGPAGGGGAARDLKGRDAATAEARHRVPTTELCRPPGPAPAPAPASVTAELPGDGRMVQLSPPALAAPAAPGRALLYSLSQPLASLGSGFFGEPDAFPMFTTNNRVKRRPSPYEMEITDGPHTKVVRRIFTNSRERWRQQNVNGAFAELRKLIPTHPPDKKLSKNEILRLAMKYINFLAKLLNDQEEEGTQRAKTGKDPVVGAGGGGGGGGGGAPPDDLL
MMSDASDMLAAALEQMDGIIAGSKALEYSNGIFDCQSPTSPFMGSLRALHLVEDLRGLLEMMETDEKEGLRCQIPDSTAETLVEWLQSQMTNGHLPGNGDVYQERLARLENDKESLVLQVSVLTDQVEAQGEKIRDLEFCLEEHREKVNATEEMLQQELLSRTSLETQKLDLMAEISNLKLKLTAVEKDRLDYEDKFRDTEGLIQEINDLRLKVSEMDSERLQYEKKLKSTKSLMAKLSSMKIKVGQMQYEKQRMEQ
MAENGDNEKMAALEAKICHQIEYYFGDFNLPRDKFLKEQIKLDEGWVPLEIMIKFNRLNRLTTDFNVIVEALSKSKAELMEISEDKTKIRRSPSKPLPEVTDEYKNDVKNRSVYIKGFPTDATLDDIKEWLEDKGQVLNIQMRRTLHKAFKGSIFVVFDSIESAKKFVETPGQKYKETDLLILFKDDYFAKKNEE

In [9]:
input_ids = tokenizer("Design a protein that mimics the shape of the numeral for the fourth ordinal position in a single, continuous structure: Begin with a vertical line, then create an angled line branching from the midpoint, extending diagonally down to the right, and finally, turn sharply to draw a horizontal line towards the left, ending at the vertical line. Ensure the protein's stability and functionality, subtly reflecting the numeral's shape. The result is <protein>", return_tensors="pt").input_ids.cuda()
n = 10
output = model.generate(
    input_ids,
    num_beams=n,
    # top_p=0.85,
    # do_sample=True,
    max_new_tokens=100,
    num_return_sequences=n,
    return_dict_in_generate=True,
    output_scores=True
)

for i in range(n):
    s = tokenizer.decode(output.sequences[i])
    mol = s[s.find('<protein>') + len('<protein>'):s.find('</protein>')]
    mol = mol.replace('<a>', '').replace(' ', '')
    print(mol)
    
    

MATDVQLADYPLMSPKAELKLEKKSGRKPRSPRDSGPQKELVIPGIVDFERIRRALRTPKPQTPGTYCFGRLSHHSFFSRHHPHPQHVTHIQDLTGKPVCVVRDFPAPLPESTVFSGCQMGIPTISVPIGDPQSNRNPQLSSEAWKKELKELASRVAFLTKEDELKKKEDTTSLNWNCKSRRPQAPGLSGTDLGAAELGIKKEQKEEPLREQGAKYSAETGRLIPASTRAVGRRRSHQGQQSQSSSRHEGVQAFLLQDQELLVLELLCRI
MMMMALSKTFGQKPVKFQLEDDGEFYMIGSEVGNYLRMFRGSLYKRYPSLWRRLATVEERKKIVASSHGKKTKPNTKDHGYTTLATSVTLLKASEVEEILDGNDEKYKAVSISTEPPTYLREQKAKRNSQWVPTLPNSSHHLDAVPCSTTINRNRMGRDKKRTFPLCFDDHDPAVIHENASQPEVLVPIRLDMEIDGQKLRDAFTWNMNEKLMTPEMFSEILCDDLDLNPLTFVPAIASAIRQQIESYPTDSILEDQSDQR
MATDVQLADYPLMSPKAELKLEKKSGRKPRSPRDSGPQKELVIPGIVDFERIRRALRTPKPQTPGTYCFGRLSHHSFFSRHHPHPQHVTHIQDLTGKPVCVVRDFPAPLPESTVFSGCQMGIPTISVPIGDPQSNRNPQLSSEAWKKELKELASRVAFLTKEDELKKKEKEQKEEPLREQGAKYSAETGRLIPASTRAVGRRRSHQGQQSQSSSRHEGVQAFLLQDQELLVLELLCRILETDLLSAIQFWLLYAPPKEKDLALGLLQTAV
MADPIMDLFDDPNLFGLDSLTDDSFNQVTQDPIEEALGLPSSLDSLDQMNQDGGGGDVGNSSASELVPPPEETAPTELSKESTAPAPESITLHDYTTQPASQEQPAQPVLQTSTPTSGLLQVSKSQEILSQGNPFMGVSATAVSSSSAGGQPPQSAPKIVILKAPPSSSVTGAHVAQIQAQGITSTAQPLVAGTAN

In [12]:
input_ids = tokenizer("A protein in family G is <protein>", return_tensors="pt").input_ids.cuda()
n = 10
output = model.generate(
    input_ids,
    num_beams=n,
    # top_p=0.85,
    # do_sample=True,
    max_new_tokens=100,
    num_return_sequences=n,
    return_dict_in_generate=True,
    output_scores=True
)

for i in range(n):
    s = tokenizer.decode(output.sequences[i])
    mol = s[s.find('<protein>') + len('<protein>'):s.find('</protein>')]
    mol = mol.replace('<a>', '').replace(' ', '')
    print(mol)

MNPLLILTFVAAALAAPFDDDDKIVGGYNCEENSVPYQVSLNSGYHFCGGSLINEQWVVSAGHCYKSRIQVRLGEHNIEVLEGNEQFINAAKIIRHPQYDRKTLNNDIMLIKLSSRAVINARVSTISLPTAPPATGTKCLISGWGNTASSGADYPDELQCLDAPVLSQAKCEASYPGKITSNMFCVGFLEGGKDSCQGDSGGPVVCNGQLQGVVSWGDGCAQKNKPGVYTKVYNYVKWIKNTIAANS
MAENGDNEKMAALEAKICHQIEYYFGDFNLPRDKFLKEQIKLDEGWVPLEIMIKFNRLNRLTTDFNVIVEALSKSKAELMEISEDKTKIRRSPSKPLPEVTDEYKNDVKNRSVYIKGFPTDATLDDIKEWLEDKGQVLNIQMRRTLHKAFKGSIFVVFDSIESAKKFVETPGQKYKETDLLILFKDDYFAKKNEERKQNKVEAKLRAKQEQEAKQKLEEDAEMKSLEEKIGCLLKFSGDLDDQTCREDLHILFSNHGEIKWIDFVR
MAEHDYHEDYGFSSFNDSSQEEHQDFLQFSKVFLPCMYLVVFVCGLVGNSLVLVISIFYHKLQSLTDVFLVNLPLADLVFVCTLPFWAYAGIHEWVFGQVMCKSLLGIYTINFYTSMLILTCITVDRFIVVVKATKAYNQQAKRMTWGKVTSLLIWVISLLVSLPQIIYGNVFNLDKLICGYHDEAISTVVLATQMTLGFFLPLLTMIVCYSVIIKTLLHAGGFQKHRSLKIIFLVMAVFLLTQMPFNLMKF
MVWKRLGALVMFPLQMIYLVVKAAVGLVLPAKLRDLSRENVLITGGGRGIGRQLAREFAERGARKIVLWGRTEKCLKETTEEIRQMGTECHYFICDVGNREEVYQTAKAVREKVGDITILVNNAAVVHGKSLMDSDDDALLKSQHINTLGQFWTTKAFLPRMLELQNGHIVCLNSVLALSAIPGAIDYCTSKASAFAFMESLTLGLLDCPGVSATTVLPFHTSTEMFQGMRV

In [6]:
model.state_dict().keys()

odict_keys(['model.embed_tokens.weight', 'model.layers.0.self_attn.q_proj.weight', 'model.layers.0.self_attn.k_proj.weight', 'model.layers.0.self_attn.v_proj.weight', 'model.layers.0.self_attn.o_proj.weight', 'model.layers.0.mlp.gate_proj.weight', 'model.layers.0.mlp.up_proj.weight', 'model.layers.0.mlp.down_proj.weight', 'model.layers.0.input_layernorm.weight', 'model.layers.0.post_attention_layernorm.weight', 'model.layers.1.self_attn.q_proj.weight', 'model.layers.1.self_attn.k_proj.weight', 'model.layers.1.self_attn.v_proj.weight', 'model.layers.1.self_attn.o_proj.weight', 'model.layers.1.mlp.gate_proj.weight', 'model.layers.1.mlp.up_proj.weight', 'model.layers.1.mlp.down_proj.weight', 'model.layers.1.input_layernorm.weight', 'model.layers.1.post_attention_layernorm.weight', 'model.layers.2.self_attn.q_proj.weight', 'model.layers.2.self_attn.k_proj.weight', 'model.layers.2.self_attn.v_proj.weight', 'model.layers.2.self_attn.o_proj.weight', 'model.layers.2.mlp.gate_proj.weight', 'mod

In [7]:
model.state_dict()['model.embed_tokens.weight']

tensor([[-1.3367e-02,  1.4038e-02,  2.5940e-03,  ..., -1.5381e-02,
         -1.2695e-02, -1.9908e-05],
        [ 1.5564e-03, -1.3962e-03,  3.3188e-04,  ..., -5.5542e-03,
          4.5586e-04, -3.9368e-03],
        [ 6.6833e-03,  9.3994e-03, -5.5847e-03,  ...,  3.7842e-03,
          5.4932e-03, -5.1880e-03],
        ...,
        [ 1.0840e-01,  5.7812e-01,  1.3906e+00,  ..., -5.0781e-01,
         -3.8281e-01, -5.4297e-01],
        [-1.4531e+00, -8.1250e-01,  1.7822e-02,  ...,  4.5703e-01,
          3.2031e-01,  1.4062e-01],
        [-1.4844e+00,  5.5859e-01,  7.0703e-01,  ...,  7.1875e-01,
         -3.1128e-02,  1.3984e+00]])

In [24]:
model.state_dict()['lm_head.weight']

tensor([[-0.0085, -0.0087,  0.0042,  ...,  0.0058, -0.0033,  0.0308],
        [-0.0299,  0.0405,  0.0179,  ..., -0.0232,  0.0239,  0.0356],
        [-0.0089,  0.0027,  0.0200,  ..., -0.0374,  0.0109, -0.0033],
        ...,
        [ 0.0046,  0.0054,  0.0134,  ..., -0.0071, -0.0082,  0.0065],
        [ 0.0189, -0.0356,  0.0223,  ...,  0.0310,  0.0098, -0.0101],
        [ 0.0026, -0.0256, -0.0008,  ...,  0.0291, -0.0260,  0.0100]],
       device='cuda:0')

In [9]:
model.state_dict()['model.layers.0.post_attention_layernorm.weight']

tensor([0.0452, 0.0515, 0.0515,  ..., 0.0522, 0.0544, 0.0515])

In [6]:
def design(prompt, sample=False):
    input_ids = tokenizer(prompt + " The result is <protein>", return_tensors="pt").input_ids.cuda()
    n = 10
    if sample:
        output = model.generate(
            input_ids,
            do_sample=True,
            top_p=0.9,
            max_new_tokens=100,
            num_return_sequences=n,
            return_dict_in_generate=True,
            output_scores=True
        )
    else:
        output = model.generate(
            input_ids,
            num_beams=n,
            # top_p=0.85,
            # do_sample=True,
            max_new_tokens=100,
            num_return_sequences=n,
            return_dict_in_generate=True,
            output_scores=True
        )

    ret = []
    for i in range(n):
        s = tokenizer.decode(output.sequences[i])
        mol = s[s.find('<protein>') + len('<protein>'):s.find('</protein>')]
        mol = mol.replace('<a>', '').replace(' ', '')
        ret.append(mol)

    return ret

In [12]:
design("Design a protein to mimic the letter 'M' using primarily alpha helices and beta sheets: Start with a vertical alpha helix for the first leg of 'M', transition into a diagonal beta sheet for the middle angles, and finish with another vertical alpha helix for the final leg, ensuring a seamless fold into the 'M' shape.")

['MSLVTVPFYQKRHRHFDQSYRNIQTRYLLDEYASKKRASTQASSQKSLSQRSSSQRASSQTSLGGTICRVCAKRVSTQEDEEQENRSRYQSLVAAYGEAKRQRFLSELAHLEEDVHLARSQARDKLDKYAIQQMMEDKLAWERHTFEERISRAPEILVRLRSHTVWERMSVKLCFTVQGFPTPVVQWYKDGSLICQAAEPGKYRIESNYGVHTLEINRADFDDTATYSAVATNAHGQVSTNAAVVVRRFRGDEEPFRSVGLP',
 'MEDSMDMDMSPLRPQNYLFGCELKADKDYHFKVDNDENEHQLSLRTVSLGAGAKDELHIVEAEAMNYEGSPIKVTLATLKMSVQPTVSLGGFEITPPVVLRLKCGSGPVHISGQHLVAVEEDAESEDEEEEDVKLLSISGKRSAPGGGSKVPQKKVKLAADEDDDDDDEEDDDEDDDDDDFDDEEAEEKAPVKKSIRDTPAKNAQKSNQNGKDSKPSSTPRSKGQESFKKQEKTPKTPKGPSSVEDIKAKMQASIEKGGSLPKVEAKFINYVK',
 'MAAHHRQNTAGRRKVQVSYVIRDEVEKYNRNGVNALQLDPALNRLFTAGRDSIIRIWSVNQHKQDPYIASMEHHTDWVNDIVLCCNGKTLISASSDTTVKVWNAHKGFCMSTLRTHKDYVKALAYAKDKELVASAGLDRQIFLWDVNTLTALTASNNTVTTSSLSGNKDSIYSLAMNQLGTIIVSGSTEKVLRVWDPRTCAKLMKLKGHTDNVKALLLNRDGTQCLSGSSDGTIRLWSLGQQRCIATYRVHDEGVWALQVN',
 'MSRVMINLILGAGVGLVLILSALLGSSDLALPHAVAALLGGGDATAHIIVWEIRLPRALAAVGVGAALGASGALMQGLLRNPLAEPGVLGVSASASLGAIIALYFGLGLLGAFTVPVLAVAGALAATLMLSVLAAARISAVQLILVGVGLSSFAGALGALAMNLAPNRFALSDMVTWMLGSVTNRSL

In [25]:
design("Design a protein with a structure resembling the letter 'M': Start with a vertical segment, then create an angled segment slanting downwards to the right, followed by a similar angled segment slanting upwards to the right, and conclude with another vertical segment. Ensure structural stability and functionality while subtly mimicking this letter's shape without direct replication.")

['MMSDASDMLAAALEQMDGIIAGSKALEYSNGIFDCQSPTSPFMGSLRALHLVEDLRGLLEMMETDEKEGLRCQIPDSTAETLVEWLQSQMTNGHLPGNGDVYQERLARLENDKESLVLQVSVLTDQVEAQGEKIRDLEFCLEEHREKVNATEEMLQQELLSRTSLETQKLDLMAEISNLKLKLTAVEKDRLDYEDKFRDTEGLIQEINDLRLKVSEMDSERLQYEKKLKSTKSLMAKLSSMKIKVGQMQYEKQRMEQ',
 'MASGDTLYIATDGSEMPAEIVELHEIEVETIPVETIETTVVGEEEEEDDDDEDGGGGDHGGGGGHGHAGHHHHHHHHHHHPPMIALQPLVTDDPTQVHHHQEVILVQTREEVVGGDDSDGLRAEDGFEDQILIPVPAPAGGDDDYIEQTLVTVAAAGKSGGGGSSSSGGGRVKKGGGKKSGKKSYLSGGAGAAGGGGADPGNKKWEQKQVQIKTLEGEFSVTMWSSDEKKDIDHETVVEEQIIGENSPPDYSEYMTGKKLPPGGIPGIDLSDPKQLAEFARM',
 'MSEAGGAGPGGCGAGAGAGAGPGALPPQPAALPPAPPQGSPCAAAAGGSGACGPATAVAAAGTAEGPGGGGSARIAVKKAQLRSAPRAKKLEKLGVYSACKAEESCKCNGWKNPNPSPTPPRADLQQIIVSLTESCRSCSHALAAHVSHLENVSEEEMNRLLGIVLDVEYLFTCVHKEEDADTKQVYFYLFKLLRKSILQRGKPVVEGSLEKKPPFEKPSIEQGVNNFVQYKFSHLPAKERQTIVELAKMFLNRINYWHLEAPSQRRLRSPNDDIS',
 'MSGALDVLQMKEEDVLKFLAAGTHLGGTNLDFQMEQYIYKRKSDGIYIINLKRTWEKLLLAARAIVAIENPADVSVISSRNTGQRAVLKFAAATGATPIAGRFTPGTFTNQIQAAFREPRLLVVTDPRADHQPLTEASYVNLPTIALCNTDSPLRYVDIAIPCNNKGA

In [26]:
design("Design a protein with a structure resembling the letter 'P': Begin with a vertical segment, then add a rounded loop extending from the top half, curving to the right and back to meet the vertical line. Ensure structural stability and functionality while subtly mimicking this letter's shape without direct replication.")

['MSGALDVLQMKEEDVLKFLAAGTHLGGTNLDFQMEQYIYKRKSDGIYIINLKRTWEKLLLAARAIVAIENPADVSVISSRNTGQRAVLKFAAATGATPIAGRFTPGTFTNQIQAAFREPRLLVVTDPRADHQPLTEASYVNLPTIALCNTDSPLRYVDIAIPCNNKGAHSVGLMWWMLAREVLRMRGTISREHPWEVMPDLYFYRDPEEIEKEEQAAAEKAVTKEEFQGEWTAPAPEFTATQPEVADWSEGVQVPSVPIQQ',
 'MKDVFVVEAARTAVARAGKQSWFTNVRADELSAIVFKELRKRIGCEDKAKQAELIDDVVWAATANAFMEQGLNQGRLAWILSDGSYDVPGCTVDRFCASGLNSISFATSTMMAGWGGDVQIAGGAQHMSHIPMGAGADVHPDLGNYMTLAAINMGYTAEIVARRFNVGREAQDQLAMESHLKCAAAQAANGGLGKAKDAIIPIMAKVPAKSAKGEVKRDGAYVATDHLAKAAAEKGEELVEFCVARDQGVRPETTMES',
 'MASGTTTTAVKIGIIGGTGLDDPEILEGRTEKYVDTPFGKPSDALILGKIKNVDCVLLARHGRQHTIMPSKVNYQANIWALKEEGCTHVIVTTACGSLREEIQPGDIVIIDQFIDRTTMRPQSFYDGSHSCARGVCHIPMAEPFCPKTREVLIETAKKLGLRCHSKGTMVTIEGPRFSSRAESFMFRTWGADVINMTTVPEVVLAKEAGICYASIAMATDYDCWKEHEEAVSVDRVLKTLKENANKAKSLLL',
 'MAEDADMRNELEEMQRRADQLADESLESTRRMLQLVEESKDAGIRTLVMLDEQGEQLERIEEGMDQINKDMKEAEKNLTDLGKFCGLCVCPCNKLKSSDAYKKAWGNNQDGVVASQPARVVDEREQMAISGGFIRRVTNDARENEMDENLEQVSGIIGNLRHMALDMGNEIDTQNRQIDRIMEKADSNKTRIDEANQRATKMLGSG',
 'M

In [28]:
design("Design a protein with a structure resembling the letter 'R': Begin with a vertical segment, add a rounded loop on the upper portion, curving rightward and reconnecting to the vertical line, and conclude with a diagonal segment extending downwards from the loop's end to the right. Ensure structural stability and functionality while subtly mimicking this letter's shape without direct replication.")

['MSEQTPAEAGAAGAREDACRDYQSSLEDLTFNSKPHINMLTILAEENLPFAKEIVSLIEAQTAKAPSSEKLPVMYLMDSIVKNVGREYLTAFTKNLVATFICVFEKVDENTRKSLFKLRSTWDEIFPLKKLYALDVRVNSLDPAWPIKPLPPNVNTSSIHVNPKFLNKSPEEPSTPGTVVSSPSISTPPIVPDIQKNLTQEQLIRQQLLAKQKQLLELQQKKLELELEQAKAQLAVSLSVQQETSNLGPGSAPSKLHVSQIPPMAVKA',
 'MTEGTCLRRRGGPYKTEPATDLGRWRLNCERGRQTWTYLQDERAGREQTGLEAYALGLDTKNYFKDLPKAHTAFEGALNGMTFYVGLQAEDGHWTGDYGGPLFLLPGLLITCHVARIPLPAGYREEIVRYLRSVQLPDGGWGLHIEDKSTVFGTALNYVSLRILGVGPDDPDLVRARNILHKKGGAVAIPSWGKFWLAVLNVYSWEGLNTLFPEMWLFPDWAPAHPSTLWCHCRQVYLPMSYCYAVRLSAAEDPLVQSLRQELYVEDFA',
 'MAEDADMRNELEEMQRRADQLADESLESTRRMLQLVEESKDAGIRTLVMLDEQGEQLERIEEGMDQINKDMKEAEKNLTDLGKFCGLCVCPCNKLKSSDAYKKAWGNNQDGVVASQPARVVDEREQMAISGGFIRRVTNDARENEMDENLEQVSGIIGNLRHMALDMGNEIDTQNRQIDRIMEKADSNKTRIDEANQRATKMLGSG',
 'MAETLSGLGDSGAAGAAALSSASSETGTRRLSDLRVIDLRAELRKRNVDSSGNKSVLMERLKKAIEDEGGNPDEIEITSEGNKKTSKRSSKGRKPEEEGVEDNGLEENSGDGQEDVETSLENLQDIDIMDISVLDEAEIDNGSVADCVEDDDADNLQESLSDSRELVEGEMKELPEQLQEHAIEDKETINNLDTSSSDFTILQEIEEPSLEPENEKILDILGETCKSEPVKEESSELEQP

In [31]:
design("Design a protein structure that looks like a triangle.")

['MAEMGSKGVTAGKIASNVQKKLTRAQEKVLQKLGKADETKDEQFEQCVQNFNKQLTEGTRLQKDLRTYLASVKAMHEASKKLNECLQEVYEPDWPGRDEANKIAENNDLLWMDYHQKLVDQALLTMDTYLGQFPDIKSRIAKRGRKLVDYDSARHHYESLQTAKKKDEAKIAKAEEELIKAQKVFEEMNVDLQEELPSLWNSRVGFYVNTFQSIAGLEENFHKEMSKLNQNLNDVLVGLEKQHGSNTFTVKAQPSDNAPAKG',
 'MAEMGSKGVTAGKIASNVQKKLTRAQEKVLQKLGKADETKDEQFEQCVQNFNKQLTEGTRLQKDLRTYLASVKAMHEASKKLNECLQEVYEPDWPGRDEANKIAENNDLLWMDYHQKLVDQALLTMDTYLGQFPDIKSRIAKRGRKLVDYDSARHHYESLQTAKKKDEAKIAKPVSLLEKAAPQWCQGKLQAHLVAQTNLLRNQAEEELIKAQKVFEEMNVDLQEELPSLWNSRVGFYVNTFQSIAGLEENFHKEMSKLNQN',
 'MMMMALSKTFGQKPVKFQLEDDGEFYMIGSEVGNYLRMFRGSLYKRYPSLWRRLATVEERKKIVASSHGKKTKPNTKDHGYTTLATSVTLLKASEVEEILDGNDEKYKAVSISTEPPTYLREQKAKRNSQWVPTLPNSSHHLDAVPCSTTINRNRMGRDKKRTFPLCFDDHDPAVIHENASQPEVLVPIRLDMEIDGQKLRDAFTWNMNEKLMTPEMFSEILCDDLDLNPLTFVPAIASAIRQQIESYPTDSILEDQSDQR',
 'MMMMMMMKKMQHQRQQQEDHANEANYARGTRLPISGEGPTSQPNSSKQTVLSWQAAIDAARQAKAAQTMSTSAPPPVGSLSQRKRQQYAKSKKQGNSSNSRPARALFCLSLNNPIRRACISIVEWKPFDIFILLAIFANCVALAIYIPFPEDDSNSTNHNLEKVEYAFLIIFTVETFLKIIAYGLLLHPNAYVRNGWN

In [32]:
design("Design a protein structure that looks like a square.")

['MMMMALSKTFGQKPVKFQLEDDGEFYMIGSEVGNYLRMFRGSLYKRYPSLWRRLATVEERKKIVASSHGKKTKPNTKDHGYTTLATSVTLLKASEVEEILDGNDEKYKAVSISTEPPTYLREQKAKRNSQWVPTLPNSSHHLDAVPCSTTINRNRMGRDKKRTFPLCFDDHDPAVIHENASQPEVLVPIRLDMEIDGQKLRDAFTWNMNEKLMTPEMFSEILCDDLDLNPLTFVPAIASAIRQQIESYPTDSILEDQSDQR',
 'MMMMMMMKKMQHQRQQQEDHANEANYARGTRLPISGEGPTSQPNSSKQTVLSWQAAIDAARQAKAAQTMSTSAPPPVGSLSQRKRQQYAKSKKQGNSSNSRPARALFCLSLNNPIRRACISIVEWKPFDIFILLAIFANCVALAIYIPFPEDDSNSTNHNLEKVEYAFLIIFTVETFLKIIAYGLLLHPNAYVRNGWNLLDFVIVIVGLFSVILEQLTKETEGGNHSSGKSGGFDVKALRAFRVLRPLRLVSGVPSLQVVLNSIIKAM',
 'MMMMALSKTFGQKPVKFQLEDDGEFYMIGSEVGNYLRMFRGSLYKRYPSLWRRLATVEERKKIVASSHGKKTKPNTKDHGYTTLATSVTLLKASEVEEILDGNDEKYKAVSISTEPPTYLREQKAKRNSQWVPTLPNSSHHLDAVPCSTTINRNRMGRDKKRTFPLWCGCIAALTLRADSALVLHFDDHDPAVIHENASQPEVLVPIRLDMEIDGQKLRDAFTWNMNEKLMTPEMFSEILCDDLDLNPLTFVPAIASAIR',
 'MMMMSLNSKQAFSMPHGGSLHVEPKYSALHSTSPGSSAPIAPSASSPSSSSNAGGGGGGGGGGGGGGGRSSSSSSSGSSGGGGSEAMRRACLPTPPSNIFGGLDESLLARAEALAAVDIVSQSKSHHHHPPHHSPFKPDATYHTMNTIPCTSAASSSSVPISHPSALAGTHHHHHHHHHHHHQPHQALEGELLE

In [33]:
design("Design a protein structure that looks like a pyramid.")

['MAAHVGSRCASRSSAGSSNTGTRSSATPGARGPGCWRETLTHSVTPARASCSSAPAARALPRKSEGATRTNGGGWKTTDMEGAHASLSLEDEVSGAGSRQRPLEGKGGETPAAEEPGSLKNYAVFATRDVSAAPEKEEEEAEGPLRAQDLRESYIQLVQGVQEWQDGCMYQGEFGLNMKLGYGKFSWPTGESYHGQFYRDHCHGLGTYMWPDGSSFTGTFYLSHREGYGTMYMKTRLFQGLYKADQRFGPGVETYPDGSQDVG',
 'MMMMALSKTFGQKPVKFQLEDDGEFYMIGSEVGNYLRMFRGSLYKRYPSLWRRLATVEERKKIVASSHGKKTKPNTKDHGYTTLATSVTLLKASEVEEILDGNDEKYKAVSISTEPPTYLREQKAKRNSQWVPTLPNSSHHLDAVPCSTTINRNRMGRDKKRTFPLCFDDHDPAVIHENASQPEVLVPIRLDMEIDGQKLRDAFTWNMNEKLMTPEMFSEILCDDLDLNPLTFVPAIASAIRQQIESYPTDSILEDQSDQR',
 'MASMLLAQRLACSFQHSYRLLVPGSRHISQAAAKVDVEFDYDGPLMKTEVPGPRSQELMKQLNIIQNAEAVHFFCNYEESRGNYLVDVDGNRMLDLYSQISSVPIGYSHPALLKLIQQPQNASMFVNRPALGILPPENFVEKLRQSLLSVAPKGMSQLITMACGSCSNENALKTIFMWYRSKERGQRGFSQEELETCMINQAPGCPDYSILSFMGAFHGRTMGCLATTHSKAIHKIDIPSFDWPIAPFPRLKYP',
 'MAAMAPALTDAAAEAHHIRFKLAPPSSTLSPGSAENNGNANILIAANGTKRKAIAAEDPSLDFRNNPTKEDLGKLQPLVASYLCSDVTSVPSKESLKLQGVFSKQTVLKSHPLLSQSYELRAELLGRQPVLEFSLENLRTMNTSGQTALPQAPVNGLAKKLTKSSTHSDHDNSTSLNGGKRALTSSALHGGEMGGSESGDLKGGM

In [40]:
design("Design a protein structure that abstractly represents the letter 'M': Construct four distinct segments - two upward slopes for the outer legs and two connecting peaks in the center. These elements should come together in a configuration that suggests the letter 'M', with each segment potentially representing separate protein chains or domains, ensuring structural feasibility and functional integrity.")

['MSLVTVPFYQKRHRHFDQSYRNIQTRYLLDEYASKKRASTQASSQKSLSQRSSSQRASSQTSLGGTICRVCAKRVSTQEDEEQENRSRYQSLVAAYGEAKRQRFLSELAHLEEDVHLARSQARDKLDKYAIQQMMEDKLAWERHTFEERISRAPEILVRLRSHTVWERMSVKLCFTVQGFPTPVVQWYKDGSLICQAAEPGKYRIESNYGVHTLEINRADFDDTATYSAVATNAHGQVSTNAAVVVRRFRGDEEPFRSVGLP',
 'MPPRPSSGELWGIHLMPPRILVECLLPNGMIVTLECLREATLITIKHELFKEARKYPLHQLLQDESSYIFVSVTQEAEREEFFDETRRLCDLRLFQPFLKVIEPVGNREEKILNREIGFAIGMPVCEFDMVKDPEVQDFRRNILNVCKEAVDLRDLNSPHSRAMYVYPPNVESSPELPKHIYNKLDKGQIIVVIWVIVSPNNDKQKYTLKINHDCVPEQVIAEAIRKKTRSMLLSSEQLKLCVLEYQGKYILKVCG',
 'MLRAKNQLFLLSPHYLRQVKESSGSRLIQQRLLHQQQPLHPEWAALAKKQLKGKNPEDLIWHTPEGISIKPLYSKRDTMDLPEELPGVKPFTRGPYPTMYTFRPWTIRQYAGFSTVEESNKFYKDNIKAGQQGLSVAFDLATHRGYDSDNPRVRGDVGMAGVAIDTVEDTKILFDGIPLEKMSVSMTMNGAVIPVLANFIVTGEEQGVPKEKLTGTIQNDILKEFMVRNTYIFPPEPSMKIIADIFEYTAKHMPKFNSISIS',
 'MFHQIWAALLYFYGIILNSIYQCPEHSQLTTLGVDGKEFPEVHLGQWYFIAGAAPTKEELATFDPVDNIVFNMAAGSAPMQLHLRATIRMKDGLCVPRKWIYHLTEGSTDLRTEGRPDMKTELFSSSCPGGIMLNETGQGYQRFLLYNRSPHPPEKCVEEFKSLTSCLDSKAFLLTPRNQEACELSNN',
 'MAENGDNEKM

In [41]:
design("Design a protein structure that captures the essence of the numeral '3': Create two consecutive loops, one above the other, each curving outward and then inward, resembling the top and bottom halves of '3'. These loops should be designed for ease of folding and structural stability, subtly reflecting the numeral's silhouette while maintaining the protein's functionality.")

['MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGGMQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGGMQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGGMQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLE',
 'MATDVQLADYPLMSPKAELKLEKKSGRKPRSPRDSGPQKELVIPGIVDFERIRRALRTPKPQTPGTYCFGRLSHHSFFSRHHPHPQHVTHIQDLTGKPVCVVRDFPAPLPESTVFSGCQMGIPTISVPIGDPQSNRNPQLSSEAWKKELKELASRVAFLTKEDELKKKEDTTSLNWNCKSRRPQAPGLSGTDLGAAELGIKKEQKEEPLREQGAKYSAETGRLIPASTRAVGRRRSHQGQQSQSSSRHEGVQAFLLQDQELLVLELLCRI',
 'MAEDADMRNELEEMQRRADQLADESLESTRRMLQLVEESKDAGIRTLVMLDEQGEQLERIEEGMDQINKDMKEAEKNLTDLGKFCGLCVCPCNKLKSSDAYKKAWGNNQDGVVASQPARVVDEREQMAISGGFIRRVTNDARENEMDENLEQVSGIIGNLRHMALDMGNEIDTQNRQIDRIMEKADSNKTRIDEANQRATKMLGSG',
 'MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGGMQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGGMQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGG

In [45]:
from tqdm import tqdm

# solubility
desc = """
Protein solubility is an important prerequisite for success in many biophysical studies and industrial applications,  including  the  production  of  the  ever  more important  protein-based  drugs  such  as  antibodies, interleukins and others. Efficient production of soluble and active proteins still remains a major challenge.Many proteins heterologously expressed in Escherichiacoliare insoluble. Solubilization attempts are plagued by relatively low success rates and often lead to theloss of biological activity. Various experimental approaches  aimed  at  improving  protein  solubility during heterologous expression include use of weak promoters, low temperature, modified growth media and fusion with solubility enhancing tags. Other methods are based on large-scale screening and ran-dom mutagenesis for solubility optimization.
""".strip()

for kind in ["low", "high"]:
    prompt = f"{desc} We designed a {kind} solubility protein and the result is <protein>"
    print(prompt)
    
    ret = []

    for i in tqdm(range(50)):
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
        output = model.generate(
            input_ids,
            top_p=0.9,
            do_sample=True,
            max_new_tokens=300,
            num_return_sequences=10,
            return_dict_in_generate=True,
            output_scores=True
        )

        for i in range(10):
            s = tokenizer.decode(output.sequences[i])
            mol = s[s.find('<protein>') + len('<protein>'):s.find('</protein>')]
            mol = mol.replace('<a>', '').replace(' ', '')
            ret.append(mol)

    print(len(ret))
    with open(f'/blob/shufxi/data/scigpt/solubility_gen/{kind}.txt', 'w') as f:
        for mol in ret:
            f.write(mol + '\n')


Protein solubility is an important prerequisite for success in many biophysical studies and industrial applications,  including  the  production  of  the  ever  more important  protein-based  drugs  such  as  antibodies, interleukins and others. Efficient production of soluble and active proteins still remains a major challenge.Many proteins heterologously expressed in Escherichiacoliare insoluble. Solubilization attempts are plagued by relatively low success rates and often lead to theloss of biological activity. Various experimental approaches  aimed  at  improving  protein  solubility during heterologous expression include use of weak promoters, low temperature, modified growth media and fusion with solubility enhancing tags. Other methods are based on large-scale screening and ran-dom mutagenesis for solubility optimization. We designed a low solubility protein and the result is <protein>


100%|██████████| 50/50 [12:49<00:00, 15.39s/it]


500
Protein solubility is an important prerequisite for success in many biophysical studies and industrial applications,  including  the  production  of  the  ever  more important  protein-based  drugs  such  as  antibodies, interleukins and others. Efficient production of soluble and active proteins still remains a major challenge.Many proteins heterologously expressed in Escherichiacoliare insoluble. Solubilization attempts are plagued by relatively low success rates and often lead to theloss of biological activity. Various experimental approaches  aimed  at  improving  protein  solubility during heterologous expression include use of weak promoters, low temperature, modified growth media and fusion with solubility enhancing tags. Other methods are based on large-scale screening and ran-dom mutagenesis for solubility optimization. We designed a high solubility protein and the result is <protein>


100%|██████████| 50/50 [12:50<00:00, 15.40s/it]


500


: 

In [7]:
desc = """
Design a protein with a structure resembling a common letter "M": start by drawing a vertical segment from the bottom upwards, establishing the left boundary of the "M". From the top of this line, curve downwards diagonally to the right, creating a smooth transition to form the first valley of the "M". Then, immediately curve upwards diagonally to the right, mirroring the previous curve but in the opposite direction, to create the central peak of the "M". Finally, complete the shape by drawing a vertical segment from the top downwards, ending at the same level as the starting point, to form the right boundary of the "M". This method combines straight lines and gentle curves to depict a stylized version of the letter "M".
"""

design(desc)

['MSLVTVPFYQKRHRHFDQSYRNIQTRYLLDEYASKKRASTQASSQKSLSQRSSSQRASSQTSLGGTICRVCAKRVSTQEDEEQENRSRYQSLVAAYGEAKRQRFLSELAHLEEDVHLARSQARDKLDKYAIQQMMEDKLAWERHTFEERISRAPEILVRLRSHTVWERMSVKLCFTVQGFPTPVVQWYKDGSLICQAAEPGKYRIESNYGVHTLEINRADFDDTATYSAVATNAHGQVSTNAAVVVRRFRGDEEPFRSVGLP',
 'MAQQAADKYLYVDKNFINNPLAQADWAAKKLVWVPSDKSGFEPASLKEEVGEEAIVELVENGKKVKVNKDDIQKMNPPKFSKVEDMAELTCLNEASVLHNLKERYYSGLIYTYSGLFCVVINPYKNLPIYSEEIVEMYKGKKRHEMPPHIYAITDTAYRSMMQDREDQSILCTGESGAGKTENTKKVIQYLAYVASSHKSKKDQGELERQLLQANPILEAFGNAKTVKNDNSSRFGKFIRINFDVNGYIVGANIETYLLEKSRAIRQAK',
 'MFHQIWAALLYFYGIILNSIYQCPEHSQLTTLGVDGKEFPEVHLGQWYFIAGAAPTKEELATFDPVDNIVFNMAAGSAPMQLHLRATIRMKDGLCVPRKWIYHLTEGSTDLRTEGRPDMKTELFSSSCPGGIMLNETGQGYQRFLLYNRSPHPPEKCVEEFKSLTSCLDSKAFLLTPRNQEACELSNN',
 'MALRVLLLTALTLCHGFNLDTENAMTFQENARGFGQSVVQLQGSRVVVGAPQEIVAANQRGSLYQCDYSTGSCEPIRLQVPVEAVNMSLGLSLAATTSPPQLLACGPTVHQTCSENTYVKGLCFLFGSNLRQQPQKFPEALRGCPQEDSDIAFLIDGSGSIIPHDFRRMKEFVSTVMEQLKKSKTLFSLMQYSEEFRIHFTFKEFQNNPNPRSLVKPITQLLGRTHTATGIRKVVRELFNITNGARKNAFKILVVITDGEKFG'

In [10]:
desc="""
Design a protein with a structure resembling a common letter "M": the process begins with a linear peptide chain extending upwards, akin to the initial vertical segment. As the protein's folding progresses, it bends towards the right, introducing a downward diagonal curve, mimicking the natural folding and bending of amino acid chains in response to intramolecular forces. This is followed by a reverse fold, where the chain curves back upwards in a diagonal trajectory, resembling the upward curve of the structure. The folding sequence concludes as the protein chain descends in a vertical alignment, parallel to the initial segment, stabilizing the overall conformation. This folding pattern is orchestrated by the protein's intrinsic properties, guiding the chain to adopt a form that mirrors the described shape through a series of strategic bends and folds.
""".strip()

design(desc, sample=True)


['MEFATSKELRIYTGKILEKVRAGERFAITHRGKPVAWLIPFENDTPEFSPLPYHEAWADIERALEASEPYYPDWHEALKESRRQK',
 'MAEGVGNEADVIVVGAGIIGSACARHILGLIQDPGLRVLVVAPGEPALQWSNTHAQWDKTHAGSYAAHYDEGRITRTIDAHPVWSRLARASIDRYRDLEVQSGVNFYTESGYLLVGRPQTKDSYYAQARHTVISEGVSAEVLDRAALGVRFPALQLPPHCEGLYQARGAGHISPRRMVAAQLACVRADSRATLLNATAQACTRQGDGQIWVLLDTGETLCAPTVIVATGAFTEPALLSGYRLPLNVHGRTVLLRDVTDLTAEAGPRAAALASL',
 'MAWQGLVLAACLLMFPSTTADCLSRCSLCAVKTQDGPKPINPLICSLQCQAALLPSEEWERCQSFLSFFTPSTLGLNDKEDLGSKSVGEGPYSELAKLSGSFLKELEKSKFLPSISTKENTLSKSLEEKLRGLSDGFREGAESELMRDAQLNDGAMETGTLYLAEEDPKEQVKRYGGFLRKYPKRSSEVAGEGDGDSMGHEDLYKRYGGFLRRIRPKLKWDNQKRYGGFLRRQFKVVTRSQEDPNAYSGELFDA',
 'MPCTRAQLQSCPTQMEEVTGCPYFLGKSGATRASQVLSQPSVPNHGFVIRESESAPGSLTLCIKTMDNITNFKIKQAGPQYFIKQGEVFSSINQLLSALTTSGFESQGRHLRLIPIDSRTCTPTFIANRGPARPGPSATPDRSATLPHRSSGLLSGSVPPPLPQTQRPNQRASPTPGSAPPPVPQTQRPTLGSRSSTLPVGTSLAAMAAAAPPAVPSTPRPGSGSPPVPSSPRPSDAASTLPRNRAAPDPYGQHVHHSGPTSGSPRAS',
 'MPWPAMVNNDDVQQLSTPSFPFKKEPHRLRLRARRRHHRSFHALRPPPGVPLPINLGNTLFRNAIIKALKEHPIPNVKPPQSSTLQRLAEHLHEKHWT

In [11]:
desc="""
Design a protein with a structure resembling a common letter "S".
""".strip()

design(desc)

['MSEAPRFFVGPEDTEINPGNYRHFFHHADEDDEEEDDSPPERQIVVGICSMAKKSKSKPMKEILERISLFKYITVVVFEEEVILNEPVENWPLCDCLISFHSKGFPLDKAVAYAKLRNPFVINDLNMQYLIQDRREVYSILQAEGILLPRYAILNRDPNNPKECNLIEGEDHVEVNGEVFQKPFVEKPVSAEDHNVYIYYPTSAGGGSQRLFRKIGSRSSVYSPESNVRKTGSYIYEEFMPTDGTDVKVYTVGPDYA',
 'MASMLLAQRLACSFQHSYRLLVPGSRHISQAAAKVDVEFDYDGPLMKTEVPGPRSQELMKQLNIIQNAEAVHFFCNYEESRGNYLVDVDGNRMLDLYSQISSVPIGYSHPALLKLIQQPQNASMFVNRPALGILPPENFVEKLRQSLLSVAPKGMSQLITMACGSCSNENALKTIFMWYRSKERGQRGFSQEELETCMINQAPGCPDYSILSFMGAFHGRTMGCLATTHSKAIHKIDIPSFDWPIAPFPRLKYP',
 'MMMMALSKTFGQKPVKFQLEDDGEFYMIGSEVGNYLRMFRGSLYKRYPSLWRRLATVEERKKIVASSHGKKTKPNTKDHGYTTLATSVTLLKASEVEEILDGNDEKYKAVSISTEPPTYLREQKAKRNSQWVPTLPNSSHHLDAVPCSTTINRNRMGRDKKRTFPLCFDDHDPAVIHENASQPEVLVPIRLDMEIDGQKLRDAFTWNMNEKLMTPEMFSEILCDDLDLNPLTFVPAIASAIRQQIESYPTDSILEDQSDQR',
 'MAAAMPLALLVLLLLGPGGWCLAEPPRDSLREELVITPLPSGDVAATFQFRTRWDSELQREGVSHYRLFPKALGQLISKYSLRELHLSFTQGFWRTRYWGPPFLQAPSGAELWVWFQDTVTDVDKSWKELSNVLSGIFCASLNFIDSTNTVTPTASFKPLGLANDTDHYFLRYAVLPREVVCTENLTPWKKLLPCSSKAGLSVLLKADRLF

In [12]:
desc="""
Design a protein with a structure resembling a common letter "R".
""".strip()

design(desc)

['MASMLLAQRLACSFQHSYRLLVPGSRHISQAAAKVDVEFDYDGPLMKTEVPGPRSQELMKQLNIIQNAEAVHFFCNYEESRGNYLVDVDGNRMLDLYSQISSVPIGYSHPALLKLIQQPQNASMFVNRPALGILPPENFVEKLRQSLLSVAPKGMSQLITMACGSCSNENALKTIFMWYRSKERGQRGFSQEELETCMINQAPGCPDYSILSFMGAFHGRTMGCLATTHSKAIHKIDIPSFDWPIAPFPRLKYP',
 'MAAAMPLALLVLLLLGPGGWCLAEPPRDSLREELVITPLPSGDVAATFQFRTRWDSELQREGVSHYRLFPKALGQLISKYSLRELHLSFTQGFWRTRYWGPPFLQAPSGAELWVWFQDTVTDVDKSWKELSNVLSGIFCASLNFIDSTNTVTPTASFKPLGLANDTDHYFLRYAVLPREVVCTENLTPWKKLLPCSSKAGLSVLLKADRLFHTSYHSQAVHIRPVCRNARCTSISWELRQTLSVVFDAFITGQGKKDWSLFRMFS',
 'MMMMALSKTFGQKPVKFQLEDDGEFYMIGSEVGNYLRMFRGSLYKRYPSLWRRLATVEERKKIVASSHGKKTKPNTKDHGYTTLATSVTLLKASEVEEILDGNDEKYKAVSISTEPPTYLREQKAKRNSQWVPTLPNSSHHLDAVPCSTTINRNRMGRDKKRTFPLCFDDHDPAVIHENASQPEVLVPIRLDMEIDGQKLRDAFTWNMNEKLMTPEMFSEILCDDLDLNPLTFVPAIASAIRQQIESYPTDSILEDQSDQR',
 'MMMMALSKTFGQKPVKFQLEDDGEFYMIGSEVGNYLRMFRGSLYKRYPSLWRRLATVEERKKIVASSHGKKTKPNTKDHGYTTLATSVTLLKASEVEEILDGNDEKYKAVSISTEPPTYLREQKAKRNSQWVPTLPNSSHHLDAVPCSTTINRNRMGRDKKRTFPLWCGCIAALTLRADSALVLHFDDHDPAVIHENASQPEV

In [7]:
desc="""
Design a protein with a structure resembling a common letter "V".
""".strip()

design(desc)

['MASMLLAQRLACSFQHSYRLLVPGSRHISQAAAKVDVEFDYDGPLMKTEVPGPRSQELMKQLNIIQNAEAVHFFCNYEESRGNYLVDVDGNRMLDLYSQISSVPIGYSHPALLKLIQQPQNASMFVNRPALGILPPENFVEKLRQSLLSVAPKGMSQLITMACGSCSNENALKTIFMWYRSKERGQRGFSQEELETCMINQAPGCPDYSILSFMGAFHGRTMGCLATTHSKAIHKIDIPSFDWPIAPFPRLKYP',
 'MMMMALSKTFGQKPVKFQLEDDGEFYMIGSEVGNYLRMFRGSLYKRYPSLWRRLATVEERKKIVASSHGKKTKPNTKDHGYTTLATSVTLLKASEVEEILDGNDEKYKAVSISTEPPTYLREQKAKRNSQWVPTLPNSSHHLDAVPCSTTINRNRMGRDKKRTFPLCFDDHDPAVIHENASQPEVLVPIRLDMEIDGQKLRDAFTWNMNEKLMTPEMFSEILCDDLDLNPLTFVPAIASAIRQQIESYPTDSILEDQSDQR',
 'MAAAMPLALLVLLLLGPGGWCLAEPPRDSLREELVITPLPSGDVAATFQFRTRWDSELQREGVSHYRLFPKALGQLISKYSLRELHLSFTQGFWRTRYWGPPFLQAPSGAELWVWFQDTVTDVDKSWKELSNVLSGIFCASLNFIDSTNTVTPTASFKPLGLANDTDHYFLRYAVLPREVVCTENLTPWKKLLPCSSKAGLSVLLKADRLFHTSYHSQAVHIRPVCRNARCTSISWELRQTLSVVFDAFITGQGKKDWSLFRMFS',
 'MMMMALSKTFGQKPVKFQLEDDGEFYMIGSEVGNYLRMFRGSLYKRYPSLWRRLATVEERKKIVASSHGKKTKPNTKDHGYTTLATSVTLLKASEVEEILDGNDEKYKAVSISTEPPTYLREQKAKRNSQWVPTLPNSSHHLDAVPCSTTINRNRMGRDKKRTFPLWCGCIAALTLRADSALVLHFDDHDPAVIHENASQPEV

In [8]:
desc="""
Design a protein with a structure resembling a common letter "n".
""".strip()

design(desc)

['MASMLLAQRLACSFQHSYRLLVPGSRHISQAAAKVDVEFDYDGPLMKTEVPGPRSQELMKQLNIIQNAEAVHFFCNYEESRGNYLVDVDGNRMLDLYSQISSVPIGYSHPALLKLIQQPQNASMFVNRPALGILPPENFVEKLRQSLLSVAPKGMSQLITMACGSCSNENALKTIFMWYRSKERGQRGFSQEELETCMINQAPGCPDYSILSFMGAFHGRTMGCLATTHSKAIHKIDIPSFDWPIAPFPRLKYP',
 'MMMMALSKTFGQKPVKFQLEDDGEFYMIGSEVGNYLRMFRGSLYKRYPSLWRRLATVEERKKIVASSHGKKTKPNTKDHGYTTLATSVTLLKASEVEEILDGNDEKYKAVSISTEPPTYLREQKAKRNSQWVPTLPNSSHHLDAVPCSTTINRNRMGRDKKRTFPLCFDDHDPAVIHENASQPEVLVPIRLDMEIDGQKLRDAFTWNMNEKLMTPEMFSEILCDDLDLNPLTFVPAIASAIRQQIESYPTDSILEDQSDQR',
 'MAAAMPLALLVLLLLGPGGWCLAEPPRDSLREELVITPLPSGDVAATFQFRTRWDSELQREGVSHYRLFPKALGQLISKYSLRELHLSFTQGFWRTRYWGPPFLQAPSGAELWVWFQDTVTDVDKSWKELSNVLSGIFCASLNFIDSTNTVTPTASFKPLGLANDTDHYFLRYAVLPREVVCTENLTPWKKLLPCSSKAGLSVLLKADRLFHTSYHSQAVHIRPVCRNARCTSISWELRQTLSVVFDAFITGQGKKDWSLFRMFS',
 'MMMMALSKTFGQKPVKFQLEDDGEFYMIGSEVGNYLRMFRGSLYKRYPSLWRRLATVEERKKIVASSHGKKTKPNTKDHGYTTLATSVTLLKASEVEEILDGNDEKYKAVSISTEPPTYLREQKAKRNSQWVPTLPNSSHHLDAVPCSTTINRNRMGRDKKRTFPLWCGCIAALTLRADSALVLHFDDHDPAVIHENASQPEV

In [11]:
desc="""
The <protein>MMMMALSKTFGQKPVKFQLEDDGEFYMIGSEVGNYLRMFRGSLYKRYPSLWRRLATVEERKKIVASSHGKKTKPNTKDHGYTTLATSVTLLKASEVEEILDGNDEKYKAVSISTEPPTYLREQKAKRNSQWVPTLPNSSHHLDAVPCSTTINRNRMGRDKKRTFPLCFDDHDPAVIHENASQPEVLVPIRLDMEIDGQKLRDAFTWNMNEKLMTPEMFSEILCDDLDLNPLTFVPAIASAIRQQIESYPTDSILEDQSDQR</protein> folds like shape of the letter "V". Combine two of this sequence to make a protein that fold like the letter "W".
"""

design(desc)

['MMMMALSKTFGQKPVKFQLEDDGEFYMIGSEVGNYLRMFRGSLYKRYPSLWRRLATVEERKKIVASSHGKKTKPNTKDHGYTTLATSVTLLKASEVEEILDGNDEKYKAVSISTEPPTYLREQKAKRNSQWVPTLPNSSHHLDAVPCSTTINRNRMGRDKKRTFPLCFDDHDPAVIHENASQPEVLVPIRLDMEIDGQKLRDAFTWNMNEKLMTPEMFSEILCDDLDLNPLTFVPAIASAIRQQIESYPTDSILEDQSDQR',
 'MMMMALSKTFGQKPVKFQLEDDGEFYMIGSEVGNYLRMFRGSLYKRYPSLWRRLATVEERKKIVASSHGKKTKPNTKDHGYTTLATSVTLLKASEVEEILDGNDEKYKAVSISTEPPTYLREQKAKRNSQWVPTLPNSSHHLDAVPCSTTINRNRMGRDKKRTFPLCFDDHDPAVIHENASQPEVLVPIRLDMEIDGQKLRDAFTWNMNEKLMTPEMFSEILCDDLDLNPLTFVPAIASAIRQQIESYPTDSILEDQSDQR',
 'MMMMALSKTFGQKPVKFQLEDDGEFYMIGSEVGNYLRMFRGSLYKRYPSLWRRLATVEERKKIVASSHGKKTKPNTKDHGYTTLATSVTLLKASEVEEILDGNDEKYKAVSISTEPPTYLREQKAKRNSQWVPTLPNSSHHLDAVPCSTTINRNRMGRDKKRTFPLCFDDHDPAVIHENASQPEVLVPIRLDMEIDGQKLRDAFTWNMNEKLMTPEMFSEILCDDLDLNPLTFVPAIASAIRQQIESYPTDSILEDQSDQR',
 'MMMMALSKTFGQKPVKFQLEDDGEFYMIGSEVGNYLRMFRGSLYKRYPSLWRRLATVEERKKIVASSHGKKTKPNTKDHGYTTLATSVTLLKASEVEEILDGNDEKYKAVSISTEPPTYLREQKAKRNSQWVPTLPNSSHHLDAVPCSTTINRNRMGRDKKRTFPLCFDDHDPAVIHENASQPEVLVPIRLDMEIDGQKL

In [13]:
desc="""
To design a protein in a 'V' shape using amino acids, envision two α-helices or β-strands as the arms, connected by a flexible loop at the vertex. Hydrophobic amino acids would line the interior for stability, while hydrophilic ones would face outward. The exact sequence and length would depend on the desired dimensions of the 'V', with the connecting loop's composition ensuring flexibility and the correct angle at the junction.
""".strip()

design(desc)

['MAPLRPLLILALLAWVALADQESCKGRCTEGFNVDKKCQCDELCSYYQSCCTDYTAECKPQVTRGDVFTMPEDEYTVYDDGEEKNNATVHEQVGGPSLTSDLQAQSKGNPEQTPVLKPEEEAPAPEVGASKPEGIDSRPETLHPGRPQPPAEEELCSGKPFDAFTDLKNGSLFAFRGQYCYELDEKAVRPGYPKLIRDVWGIEGPIDAAFTRINCQGKTYLFKGSQYWRFEDGVLDPDYPRNISDGFDGIPDNVDAALALPAHSYS',
 'MASGADSKGDDLSTAILKQKNRPNRLIVDEAINEDNSVVSLSQPKMDELQLFRGDTVLLKGKKRREAVCIVLSDDTCSDEKIRMNRVVRNNLRVRLGDVISIQPCPDVKYGKRIHVLPIDDTVEGITGNLFEVYLKPYFLEAYRPIRKGDIFLVRGGMRAVEFKVVETDPSPYCIVAPDTVIHCEGEPIKREDEEESLNEVGYDDIGGCRKQLAQIKEMVELPLRHPALFKAIGVKPPRGILLYGPPGTGKTLIARAVANETGAFFFLIN',
 'MGLAMEHGGSYARAGGSSRGCWYYLRYFFLFVSLIQFLIILGLVLFMVYGNVHVSTESNLQATERRAEGLYSQLLGLTASQSNLTKELNFTTRAKDAIMQMWLNARRDLDRINASFRQCQGDRVIYTNNQRYMAAIILSEKQCRDQFKDMNKSCDALLFMLNQKVKTLEVEIAKEKTICTKDKESVLLNKRVAEEQLVECVKTRELQHQERQLAKEQLQKVQALCLPLDKDKFEMDLRNLWRDSIIPRSLDNLGYNLYHPLGS',
 'MYDRPVASLKYVGPQKAARLTRLGIQTVGDLLWHFPRRYEDRRQLKELAAATPGEIITVQVTIKAWEEKEIRPRLRLFRALVQGRQGTGYALWFNQPFIKRQLPAGSEVILTGKVNYRGYVPEIQVSDYEALGQGDSGLHTGRIVPFYPLTAGLSQRWLRLIVYLALEEVQGELPEILPPSLCQ

In [14]:
desc="""
For a protein to fold into an 'O' shape, consider a structure where a single polypeptide chain forms a circular or toroidal arrangement. This could be achieved by designing a sequence that encourages the formation of a coiled coil that loops back onto itself, with the N- and C-termini positioned closely to facilitate bonding or interaction. Stabilizing interactions, such as disulfide bridges between cysteine residues near the termini, would help to maintain the circular shape. Hydrophobic residues would be oriented internally to stabilize the structure, with hydrophilic residues on the outer surface to interact with the aqueous environment.
"""
design(desc)

['MKTLQSTLLLLLLVPLIKPAPPTQQDSRIIYDYGTDNFEESIFSQDYEDKYLDGKNIKEKETVIIPNEKSLQLQKDEAITPLPPKKENDEMPTCLLCVCLSGSVYCEEVDIDAVPPLPKESAYLYARFNKIKKLTAKDFADIPNLRRLDFTGNLIEDIEDGTFSKLSLLEELSLAENQLLKLPVLPPKLTLFNAKYNKIKSRGIKANAFKKLNNLTFLYLDHNALESVPLNLPESLRVIHLQFNNIASITDDTFCKANDTSYIRDRIEEIRLEGNPIVLGKHP',
 'MAGELADKKDRDVSPSKEERKRSRTPDRERDRDRDRKSSPSKDRKRHRSRDRRRGGSRSRSRSRSKSAERERRHKERDREKERDRNKKDRDRDKDGHRRDKDRKRSSLSPGRGKDFKSRKDRDPKKDGEDEHGDKKPKIQPLSLEELLAKKKAEEEAEAKPKFLSKAEREAEALKRRQQEVEERQRMLEEERKKRKQFQDLGRKMLEDPQERERRERRERMERETNGNEDEEGRQKIREEKDKSKELHAIKERYLGGIKKRRRTRHLNDRKFVFEWDAS',
 'MMRVCWLVRQDSRHQRIRLPHLEAVVIGRGPETKITDKKCSRQQVQLKAECNKGYVKVKQVGVNPTSIDSVVIGKDQEVKLQPGQVLHMVNELYPYIVEFEEEAKNPGLETHRKRKRSGNSDSIERDAAQEAEAGTGLEPGSNSGQCSVPLKKGKDAPIKKESLGHWSQGLKISMQDPKMQVYKDEQVVVIKDKYPKARYHWLVLPWTSISSLKAVAREHLELLKHMHTVGEKVIVDFAGSSKLRFRLGYHAIPSMSHVHLHVI',
 'MEREKFQQKALKQTKQKKSKSAEFLMVKEDREATEGTGNPAFNMSSPDLSACQTAEKKVIRHDMPDRTLAAHQQKFRLPASAEPKGNEYGRNYFDPLMDEEINPRQCATEVSREVCLMAKGEKAESGVDNTAGLSEKETQKNPRAPMDEADVKSAAH

In [15]:
desc="""
To describe a protein folding into the shape of the letter 'O', envision a continuous loop formed by the polypeptide chain. This loop would create a circular or toroidal structure, with the chain curving around to form a closed ring, representing the 'O'.
"""

design(desc)

['MEHPSKMEFFQKLGYDREDVLRVLGKLGEGALVNDVLQELIRTGSRPGALEHLAAPRLVPRGSCGALDSAQRGPGTALEEDFGALASSLRPIVIDGSNVAMSHGNKETFSCRGIKLAVDWFRDRGHTYIKVFVPSWRKDPPRADTPIREQHVLAELERQAVLVYTPSRKVHGKRLVCYDDRYIVKVAYEQDGVIVSNDNYRDLQSENPEWKWFIEQRLLMFSFVNDRFMPPDDPLGRHGPSLTNFLSRKPKPPEPSWQHC',
 'MEQRRPWPRALEVDSRSVVLLSVVWVLLAPPAAGMPQFSTFHSENRDWTFNHLTVHQGTGAVYVGAINRVYKLTGNLTIQVAHKTGPEEDNKSCYPPLIVQPCSEVLTLTNNVNKLLIIDYSENRLLACGSLYQGVCKLLRLDDLFILVEPSHKKEHYLSSVNKTGTMYGVIVRSEGEDGKLFIGTAVDGKQDYFPTLSSRKLPRDPESSAMLDYELHSDFVSSLIKIPSDTLALVSHFDIFYIYGFASGGFVYFLTVQPETPEGVAINSAG',
 'MAGELADKKDRDVSPSKEERKRSRTPDRERDRDRDRKSSPSKDRKRHRSRDRRRGGSRSRSRSRSKSAERERRHKERDREKERDRNKKDRDRDKDGHRRDKDRKRSSLSPGRGKDFKSRKDRDPKKDGEDEHGDKKPKIQPLSLEELLAKKKAEEEAEAKPKFLSKAEREAEALKRRQQEVEERQRMLEEERKKRKQFQDLGRKMLEDPQERERRERRERMERETNGNEDEEGRQKIREEKDKSKELHAIKERYLGGIKKRRRTRHLNDRKFVFEWDAS',
 'MLWQKPTAPEQAPAPARPYQGVRVKEPVKELLRRKRGHASSGAAPAPTAVVLPHQPLATYTTVGPSCLDMEGSVSAVTEEAALCAGWLSQPTPATLQPLAPWTPYTEYVPHEAVSCPYSADMYVQPVCPSYTVVGPSSVLTYASPPLITNVTTRSSATPAVGPPLEGPEHQA